Map Reduce 


list of docs
for doc in list of docs | prmopt | llm 

for response in list of llm response | put them all together

final doc | prmopt | llm

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores import Chroma ## vector store
from langchain.vectorstores import FAISS ## vector store
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts import ChatPromptTemplate

cache_dir = LocalFileStore("../../db/vector-store/")

llm = ChatOpenAI(
    temperature=0.1
)
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./file/little_prince.txt")

docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()
cached_embeddins = CacheBackedEmbeddings.from_bytes_store(embeddings,cache_dir)
vectorstore = FAISS.from_documents(docs,cached_embeddins)

retriever = vectorstore.as_retriever()


final_p = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",),
        ("human", "{question}"),
    ]
)

map_doc_prmopt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",),
        ("human", "{question}")
])

map_doc_chain = map_doc_prmopt | llm 

def map_docs(inputs):
    print(inputs)
    documents = inputs['documents']
    question = inputs['question']
    return "\n\n".join(map_doc_chain.invoke({
        "context" : doc.page_content,
        "question" : question
    }).content for doc in documents )

map_chain = (
    {
        "documents" : retriever, 
        "question" : RunnablePassthrough()
     } 
| RunnableLambda(map_docs)
)

chain = {"context": map_chain ,"question" : RunnablePassthrough()} | final_p | llm


result = chain.invoke("Describe fox")

print(result.content)

{'documents': [Document(page_content='The fox gazed at the little prince, for a long time.\n"Please-\ntame me!" he said.\n"I want to, very much," the little prince replied. "But I have not much time. I have friends to discover, and a great many things to understand."\n"One only understands the things that one tames," said the fox. "Men have no more time to understand anything. They buy things all ready made at the shops. But there is no shop anywhere where one can buy friendship, and so men have no friends any more. If you want a friend, tame me . . ."\n"What must I do, to tame you?" asked the little prince.\n"You must be very patient," replied the fox. "First you will sit down at a little distance from me--like that--in the grass. I shall look at you out of the corner of my eye, and you will say nothing. Words are the source of misunderstandings. But you will sit a little closer to me, every day . . ."\nThe next day the little prince came back.\n"It would have been better to come back